# Payoff d'une butterfly call sur SPY

TP sur la structure butterfly : long K1, short 2×K2, long K3 (K1 < K2 < K3) sur SPY.
Objectifs :
1. Fixer un spot de référence (historique SPY sur un an).
2. Illustrer le payoff "tiroir" borné et centré sur K2.
3. Mettre en évidence la symétrie et le cap de gain/perte.

In [ ]:
%pip -q install yfinance matplotlib pandas numpy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("seaborn-v0_8-darkgrid")

def payoff_call(spot: float, strike: float) -> float:
    return max(spot - strike, 0.0)

def payoff_butterfly(spot: float, k1: float, k2: float, k3: float) -> float:
    return payoff_call(spot, k1) - 2 * payoff_call(spot, k2) + payoff_call(spot, k3)

def fetch_spy_history(period="1y", interval="1d") -> pd.Series:
    data = yf.download("SPY", period=period, interval=interval, progress=False)
    if data.empty or "Close" not in data:
        raise RuntimeError("Impossible de récupérer les prix SPY")
    return data["Close"]

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
k1 = spot_ref * 0.95
k2 = spot_ref
k3 = spot_ref * 1.05
spot_ref, k1, k2, k3

## Évolution du sous-jacent (SPY)
Clôtures sur un an et repère du spot de référence (dernier close).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()

## Payoff de la butterfly call
Gain maximal autour de K2, décroissance symétrique vers 0 aux extrêmes.

In [ ]:
S_grid = np.linspace(spot_ref * 0.5, spot_ref * 1.5, 200)
payoffs = [payoff_butterfly(S, k1, k2, k3) for S in S_grid]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(S_grid, payoffs, color="navy", label="Payoff butterfly long")
ax.axvline(k1, color="gray", linestyle=":", label=f"K1 = {k1:.2f}")
ax.axvline(k2, color="gray", linestyle="--", label=f"K2 = {k2:.2f}")
ax.axvline(k3, color="gray", linestyle=":", label=f"K3 = {k3:.2f}")
ax.set_xlabel("Spot S")
ax.set_ylabel("Payoff à maturité")
ax.set_title("Butterfly call sur SPY : payoff vs spot")
ax.legend()
plt.show()